In [1]:
import tensorflow as tf
import xlrd

In [2]:
import pandas as pd

In [3]:
d = pd.read_table('dataset.txt')

In [4]:
d[d.isnull().values==True]
d = d.fillna(0)

In [5]:
b = pd.read_table('2019MVP_pred.txt')
b = b.fillna(0)
b.head()

,year,player,age,g,mp,per,ts,3par,ftr,orb,...,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp,game_win,mvp_label
0,2019,James Harden,29,78,2867,30.6,0.616,0.539,0.449,2.5,...,11.4,3.8,15.2,0.254,10.5,1.1,11.7,9.9,53,1
1,2019,Giannis Antetokounmpo,24,72,2358,30.9,0.644,0.163,0.550,7.3,...,8.9,5.5,14.4,0.292,5.7,5.0,10.8,7.6,60,1


In [6]:
import numpy as np

In [7]:
temp = d.values[:,0]
temp1 = b.values[:,0]

In [8]:
temp = np.reshape(temp, (-1,1))
temp1 = np.reshape(temp1, (-1,1))
a = np.hstack((temp, d.values[:,2:]))
t = np.hstack((temp1, b.values[:,2:]))
# t = np.hstack((temp1, b.values[:, 2:16]))

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

In [10]:
transform = Normalizer().fit(a[:,1:25])
new_a = transform.transform(a[:,1:25])
transform = Normalizer().fit(t[:,1:25])
new_t = transform.transform(t[:,1:25])

In [11]:
a_ = np.hstack((a[:,0].reshape(-1,1), new_a))
a = np.hstack((a_, a[:,25].reshape(-1,1)))
a = np.hstack((a, d.values[:,-1].reshape(-1,1)))

In [12]:
t_ = np.hstack((t[:,0].reshape(-1,1), new_t))
t = np.hstack((t_, t[:,25].reshape(-1,1)))
t = np.hstack((t, b.values[:,-1].reshape(-1,1)))

In [13]:
x = tf.placeholder(dtype=tf.float32, shape=([None, 25]), name='input')
y_true = tf.placeholder(dtype=tf.float32, shape=([None,1]), name='mvp')

class Helper():
    
    def __init__(self, data, test_data):
        self.i = 0
        self.all_train_batches = data
        self.test_images = test_data[:,:25]
        self.test_labels = test_data[:,25]
    
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

# def conv2d(x, W):
#     return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# def max_pool_2by2(x):
#     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
#                           strides=[1, 2, 2, 1], padding='SAME')

# def convolutional_layer(input_x, shape):
#     W = init_weights(shape)
#     b = init_bias([shape[3]])
#     return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b



In [14]:
s = [0]*31
sum_ = 0
print(len(temp))
for year in temp:
    s[int(year)-1986] += 1
for i in range(len(s)):
    s[i] += sum_
    sum_ = s[i]
    
tt = [0]*3
sum_ = 0
print(len(temp1))
for year in temp1:
    tt[int(year)-2017] += 1
for i in range(len(tt)):
    tt[i] += sum_
    sum_ = tt[i]


out1 = normal_full_layer(x,500)
layer1 = tf.tanh(out1)

out2 = normal_full_layer(layer1, 100)
layer2 = tf.tanh(out2)

out3 = normal_full_layer(layer2, 1)
y_pred = tf.sigmoid(out3)

mse = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_true,predictions=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00012)
train = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5000):
        loopNum = i%20
        if loopNum == 0:
            _, loss, y_hat = sess.run([train,mse,y_pred], feed_dict={x: a[:s[loopNum], 1:26], y_true: a[:s[loopNum], 26].reshape(-1,1)})
        else:
            _, loss, y_hat = sess.run([train,mse, y_pred], feed_dict={x: a[s[loopNum-1]:s[loopNum], 1:26], y_true: a[s[loopNum-1]:s[loopNum], 26].reshape(-1,1)})
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('loss is:{:f}'.format(loss))
            # print(y_hat)
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            accuracy, y_hat2 = sess.run([acc,y_pred],feed_dict={x:t[:,1:26],y_true:t[:,26].reshape(-1,1)})
            print('\n')
    print(accuracy)

15059
2
Currently on step 0
loss is:0.255795


Currently on step 100
loss is:0.005794


Currently on step 200
loss is:0.003573


Currently on step 300
loss is:0.002733


Currently on step 400
loss is:0.001846


Currently on step 500
loss is:0.000941


Currently on step 600
loss is:0.000519


Currently on step 700
loss is:0.000337


Currently on step 800
loss is:0.000238


Currently on step 900
loss is:0.000179


Currently on step 1000
loss is:0.000139


Currently on step 1100
loss is:0.000111


Currently on step 1200
loss is:0.000091


Currently on step 1300
loss is:0.000076


Currently on step 1400
loss is:0.000064


Currently on step 1500
loss is:0.000055


Currently on step 1600
loss is:0.000047


Currently on step 1700
loss is:0.000041


Currently on step 1800
loss is:0.000036


Currently on step 1900
loss is:0.000032


Currently on step 2000
loss is:0.000028


Currently on step 2100
loss is:0.000025


Currently on step 2200
loss is:0.000023


Currently on step 2300
loss is:0.00002

In [15]:
b.values[np.argpartition(y_hat2[:,0], -2)[-2:],0:2]

array([[2019, 'Giannis Antetokounmpo'],
       [2019, 'James Harden']], dtype=object)

In [16]:
y_hat2[np.argpartition(y_hat2[:,0], -2)[-2:]]

array([[ 0.98552716],
       [ 0.98597878]], dtype=float32)

In [17]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())

#     for i in range(5000):
#         loopNum = i%20
#         sess.run(train, feed_dict={x: batch[0], y_true: batch[1]})
        
#         # PRINT OUT A MESSAGE EVERY 100 STEPS
#         if i%100 == 0:
            
#             print('Currently on step {}'.format(i))
#             print('Accuracy is:')
#             # Test the Train Model
#             matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))

#             acc = tf.reduce_mean(tf.cast(matches,tf.float32))

#             print(sess.run(acc,feed_dict={x:,y_true:ch.test_labels,hold_prob:1.0}))
#             print('\n')


In [19]:
print("2019 NBA MVP predicted by the model: " + b.values[np.argmax(y_hat2),1])

2019 NBA MVP predicted by the model: James Harden
